In [1]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

     |████████████████████████████████| 4.3 MB 5.0 MB/s 


In [ ]:
!pip install -q tf-models-official

     |████████████████████████████████| 1.6 MB 5.2 MB/s 
     |████████████████████████████████| 37.1 MB 50 kB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 636 kB 48.4 MB/s 
     |████████████████████████████████| 99 kB 8.6 MB/s 
     |████████████████████████████████| 211 kB 44.6 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 679 kB 56.4 MB/s 
     |████████████████████████████████| 352 kB 55.5 MB/s 
     |████████████████████████████████| 1.2 MB 54.5 MB/s 


In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
import pandas as pd

In [ ]:
df1=pd.read_csv('data_ryan_2.csv')
df2=pd.read_csv('data_natgeo_2.csv')
#df3=pd.read_csv('data_disney_2.csv')
#df4=pd.read_csv('data_sports_2.csv')

In [ ]:
df1['label']=1
df2['label']=0
#df3['label']=1
#df4['label']=0

In [ ]:
#df3[:500]

In [ ]:
df=df1.append(df2)
#df=df1.append([df2,df3[:500],df4[:500]])
df.reset_index(drop=True)
df.head(5)

,title,channel_title,desc,date,tags,liked,disliked,views,filtered_title,label
0,Easy DIY Science Experiment for Kids Rainbow S...,Ryan's World,Easy DIY Science Experiment for Kids Rainbow S...,2021-01-24T13:00:02Z,"[""Ryan's World"", 'Ryan ToysReview', 'science e...",1503,399,200970,Easy Science Experiment for s Rainbow Snowsto...,1
1,Ryan play with Giant Soccer Ball and Learn abo...,Ryan's World,Ryan play with Giant Soccer Ball and Learn abo...,2021-01-23T13:00:32Z,"[""Ryan's World"", 'force', 'force and motion', ...",2434,626,331667,with Giant Soccer Ball and Learn about Force...,1
2,Ryan hides the Golden Egg from King Collectors...,Ryan's World,Ryan hides the Golden Egg from King Collectors...,2021-01-22T13:00:22Z,"[""Ryan's World"", 'Pretend play', 'Combo Panda'...",2469,605,332385,hides the Golden Egg from King Collectors wit...,1
3,Dancing Corn Easy DIY Science Experiments for ...,Ryan's World,Dancing Corn Easy DIY Science Experiments for ...,2021-01-20T13:00:13Z,"[""Ryan's World"", 'Ryan ToysReview', 'dancing c...",7228,2360,1587332,Dancing Corn Easy Science Experiments for to...,1
4,Ryan does DIY Science Experiments with Emma an...,Ryan's World,Ryan does DIY Science Experiments with Emma an...,2021-01-18T13:00:29Z,"[""Ryan's World"", 'Ryan ToysReview', 'DIY scien...",16552,6395,4411698,does Science Experiments with Emma and Kate ...,1


In [ ]:
df.shape

(3804, 10)

In [ ]:
!pip install transformers

import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

     |████████████████████████████████| 2.6 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 3.3 MB 49.4 MB/s 


Importing pre-trained DistilBERT model and tokenizer

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch_1 = df
X=df['filtered_title']
Y=df['label']

Tokenization


In [ ]:
tokenized = X.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Padding

After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

(3804, 27)

Masking

If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(3804, 27)

Processing with DistilBERT

In [ ]:
input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
features.shape

(3804, 768)

In [ ]:
labels = Y
labels.shape

(3804,)

In [ ]:
#train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
# split into train validation test sets
train_ratio = 0.60
validation_ratio = 0.20
test_ratio = 0.20

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=1) 

#print(x_train, x_val, x_test)


In [ ]:
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(x_test, y_test)

0.9724047306176085

In [ ]:
roc_auc_score(y_test, lr_clf.predict_proba(x_test)[:,1])

0.9962837091069849

In [ ]:
confusion_matrix(y_test, lr_clf.predict(x_test))

array([[371,   6],
       [ 15, 369]])

In [ ]:
roc_auc_score(y_val, lr_clf.predict_proba(x_val)[:,1])

0.995551404455514

In [ ]:
confusion_matrix(y_val, lr_clf.predict(x_val))

array([[354,  11],
       [ 16, 380]])